# QuantX SDK はじめの一歩

Special thanks, shinseitaroさん!

このドキュメントは shinseitaroさんに作成いただいたものをSmartTradeにて編集させていただきました。ありがとうございます！

このインストラクションは未完成です。メモ程度なので、わかりにくいと思います。ご意見いただければ書き直しますので教えてくださいm(_ _)m

### quantx sdk 

+ SDK のインストールとかはこちらから
https://factory.quantx.io/quantx-sdk/readme/readme.html

+ api key は、ログイン後
https://factory.quantx.io/settings/token
から取得

### 事前準備
`api_key.py` を用意

```python
public_key = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
secret_key = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
```

### todo  

+ upload_source_fileを使う時に動的にパラメータを渡す方法。恐らくjinja2の機能の方にあると思うのでそっちを探す


In [ ]:
# cell の大きさを100％にする
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import time
from jinja2 import Template
import pandas as pd
from quantx_sdk import QX

# アルゴリズムを書いたファイル。ファイルからloadしても実行できますよ、という話です。
#import test_algorithm_source


## 同じディレクトリに api_key.py を作って下記のように書いておくと 下記のようにimportして使える。直打ちでも大丈夫です。
#public_key = "your public key"
#secret_key = "your secret key"
import api_key



## はじめの一歩
### sample1 

public_keyとsecret_keyは各自ご自身のキーで置き換えてください

In [ ]:
## 新規プロジェクト作成
qx = QX(api_key.public_key, api_key.secret_key)

## 新規プロジェクト名。５文字以上の適当な文字列
new_name = "an new project"
## 新規プロジェクトを作成。
prj = qx.new_project(new_name)

## 新規でプロジェクトを作成すると、ユニークなハッシュ番号が発行されますので、それを変数に格納しておきます。
## このハッシュ番号を使って一つのプロジェクトを使い回します。
## （そうしないと新しいプロジェクトをいくつも作成してしまうことになります）
prjct_hash = prj.project_hash

## プロジェクトを新規作成すると、サンプルコードが自動生成されます。
## プロジェクトに書いているコードを取得するには .source() を使います。
#prj.source()

## プロジェクトの新規作成時のパラメータを確認
bt_parameter = prj.current_backtest_params()

## backtest の実行. backtest object を返す
bt = prj.backtest(bt_parameter)

## backtest result インスタンスを作成
join = bt.join()

## 結果のサマリーを取得
summary = join.summary()

In [ ]:
## summary を見ると何も結果無し。
## これは、最初に自動生成されるサンプルコードでは、取引を実行する部分が記述されていないからです。
## 0 が並んでいるなら正解です。
summary

## はじめの一歩その２

sample１は常に新規作成されてしまいます。
何度か実行してみると、自分のプロジェクトに同じものが新規作成されていると思います

https://factory.quantx.io/projects

これを回避するために、新規作成したプロジェクトのハッシュ番号をプロジェクトオブジェクトに渡して一つのプロジェクトを使い回すようにします。
（既存のプロジェクトハッシュでも可。その場合は上書きされるので注意）


In [ ]:
#prj = qx.new_project("2018-07-21-ver2")
## プロジェクトのハッシュ番号を取得
prjct_hash = prj.project_hash


In [ ]:
prjct_hash

In [ ]:
#簡単なバックテストコード
source = """

def initialize(ctx):
    # 設定
    ctx.logger.debug("initialize() called")
    ctx.configure(
      target="jp.stock.daily",
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols": [
            "jp.stock.1305",
            "jp.stock.9984",
            "jp.stock.9983",
          ],
          "columns": [
            #"open_price_adj",    # 始値(株式分割調整後)
            #"high_price_adj",    # 高値(株式分割調整後)
            #"low_price_adj",     # 安値(株式分割調整後)
            "close_price",        # 終値
            "close_price_adj",    # 終値(株式分割調整後) 
            "volume_adj",         # 出来高
            "txn_volume",         # 売買代金
          ]
        }
      }
    )

    def _mavg_signal(data):
        m25 = data["close_price_adj"].fillna(method='ffill').rolling(window=25, center=False).mean()
        m75 = data["close_price_adj"].fillna(method='ffill').rolling(window=75, center=False).mean()
        ratio = m25 / m75

        buy_sig = ratio[(ratio > 1.02) & (ratio < 1.05)]
        sell_sig = ratio[ratio < 0.95]
        return {
            "mavg_25:price": m25,
            "mavg_75:price": m75,
            #"mavg_ratio:ratio": ratio,
            "buy:sig": buy_sig,
            "sell:sig": sell_sig,
        }

    # シグナル登録
    ctx.regist_signal("mavg_signal", _mavg_signal)

def handle_signals(ctx, date, current):
    '''
    current: pd.DataFrame
    '''

    done_syms = set([])
    for (sym,val) in ctx.portfolio.positions.items():
        returns = val["returns"]
        #ctx.logger.debug("%s %f" % (sym, returns))
        if returns < -0.03:
          sec = ctx.getSecurity(sym)
          sec.order(-val["amount"], comment="損切り(%f)" % returns)
          done_syms.add(sym)
        elif returns > 0.05:
          sec = ctx.getSecurity(sym)
          sec.order(-val["amount"], comment="利益確定売(%f)" % returns)
          done_syms.add(sym)


    buy = current["buy:sig"].dropna()
    for (sym,val) in buy.items():
        if sym in done_syms:
          continue
        
        sec = ctx.getSecurity(sym)
        sec.order(sec.unit() * 1, comment="SIGNAL BUY")
        #ctx.logger.debug("BUY: %s,  %f" % (sec.code(), val))
        pass

    sell = current["sell:sig"].dropna()
    for (sym,val) in sell.items():
        if sym in done_syms:
          continue

        sec = ctx.getSecurity(sym)
        sec.order(sec.unit() * -1, comment="SIGNAL SELL")
        #ctx.logger.debug("SELL: %s,  %f" % (sec.code(), val))
        pass

"""

In [ ]:
## hash番号を与えてプロジェクトオブジェクトを作ります
prj = qx.project(prjct_hash)

## プロジェクトにサンプルを渡します。
## 一つは、上記のようにコードを書いて文字列として渡す方法
prj.upload_source(source)

## もうひとつは、アルゴリズムを書いたファイルをUploadする方法
## このnotebookと同じディレクトリに保存したファイル "test_algorithm_source.py" を読み込む
#prj.upload_source_file("test_algorithm_source.py")

## backtest の実行. backtest object を返す
bt = prj.backtest(bt_parameter)

## backtest result インスタンスを作成
join = bt.join()

## 結果のサマリーを取得
summary = join.summary()

In [ ]:
prj.project_hash

## メモ

ウェブ上でプロジェクトを確認すると、最後にUploadしたアルゴリズムで上書きされているはずです。確認してみてください。


## はじめの一歩その３

アルゴリズムをテンプレート化

銘柄を動的に変えて評価していきましょう。

ソーステンプレートの

```python
{{ symbols }}
```
部分をこのようにすることで動的に評価できますよ。

```python
source = Template(original_source).render({'symbols': ['jp.stock.8303','jp.stock.8304','jp.stock.2651']},)
```




In [ ]:
original_source = """

def initialize(ctx):
    # 設定
    ctx.logger.debug("initialize() called")
    ctx.configure(
      target="jp.stock.daily",
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols":{{ symbols }},
          "columns": [
            #"open_price_adj",    # 始値(株式分割調整後)
            #"high_price_adj",    # 高値(株式分割調整後)
            #"low_price_adj",     # 安値(株式分割調整後)
            "close_price",        # 終値
            "close_price_adj",    # 終値(株式分割調整後) 
            "volume_adj",         # 出来高
            "txn_volume",         # 売買代金
          ]
        }
      }
    )

    def _mavg_signal(data):
        m25 = data["close_price_adj"].fillna(method='ffill').rolling(window=25, center=False).mean()
        m75 = data["close_price_adj"].fillna(method='ffill').rolling(window=75, center=False).mean()
        ratio = m25 / m75

        buy_sig = ratio[(ratio > 1.02) & (ratio < 1.05)]
        sell_sig = ratio[ratio < 0.95]
        return {
            "mavg_25:price": m25,
            "mavg_75:price": m75,
            #"mavg_ratio:ratio": ratio,
            "buy:sig": buy_sig,
            "sell:sig": sell_sig,
        }

    # シグナル登録
    ctx.regist_signal("mavg_signal", _mavg_signal)

def handle_signals(ctx, date, current):
    '''
    current: pd.DataFrame
    '''

    done_syms = set([])
    for (sym,val) in ctx.portfolio.positions.items():
        returns = val["returns"]
        #ctx.logger.debug("%s %f" % (sym, returns))
        if returns < -0.03:
          sec = ctx.getSecurity(sym)
          sec.order(-val["amount"], comment="損切り(%f)" % returns)
          done_syms.add(sym)
        elif returns > 0.05:
          sec = ctx.getSecurity(sym)
          sec.order(-val["amount"], comment="利益確定売(%f)" % returns)
          done_syms.add(sym)


    buy = current["buy:sig"].dropna()
    for (sym,val) in buy.items():
        if sym in done_syms:
          continue
        
        sec = ctx.getSecurity(sym)
        sec.order(sec.unit() * 1, comment="SIGNAL BUY")
        #ctx.logger.debug("BUY: %s,  %f" % (sec.code(), val))
        pass

    sell = current["sell:sig"].dropna()
    for (sym,val) in sell.items():
        if sym in done_syms:
          continue

        sec = ctx.getSecurity(sym)
        sec.order(sec.unit() * -1, comment="SIGNAL SELL")
        #ctx.logger.debug("SELL: %s,  %f" % (sec.code(), val))
        pass

"""

In [ ]:
## 上のソースは jinja2 というテンプレートライブラリを使っています
## "symbols":{{ symbols }},のところにある{{ symbols }} に銘柄を動的に渡せるようにします。

tickers = [2651, 2652, 2653, ]
sim = 'jp.stock.{}'
symbols = [sim.format(i) for i in tickers]
source2 = Template(original_source).render({'symbols': symbols})

## ここで source2 を実行するとtickers を含んだサンプルコードを示すことができます。
## source2 

In [ ]:
## source2 をロードして実行してみます。
## 先ほど作ったプロジェクトを再利用するために、hash番号を与えてプロジェクトオブジェクトを得ます。
prj = qx.project(prjct_hash)
prj.upload_source(source2)
bt = prj.backtest(bt_parameter)
join = bt.join()
summary = join.summary()


In [ ]:
## 結果を出してみましょう。
summary

In [ ]:
## 銘柄ごとのサマリーを見るとこんな感じですよ
join.symbol_summary()